In [10]:
#!pip install tensorflow
#!pip install tqdm
#!pip install --upgrade music21
#!pip install pandas
#!pip install glob
#!pip install KERAS
#!pip install matplotlib

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import glob
from tqdm import tqdm
from music21 import *
from tensorflow.python.ops import control_flow_ops
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [11]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
notes = []
for file in glob.glob("*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [3]:
n_vocab=len(set(notes))

In [4]:
sequence_length = 500
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [5]:
print('InputLength:',len(network_input))

InputLength: 6453


In [12]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(256))
model.add(Dense(n_vocab,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['categorical_crossentropy','accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 500, 256)          264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 500, 256)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 155)               39835     
Total params: 829,339
Trainable params: 829,339
Non-trainable params: 0
_________________________________________________________________


In [ ]:
filepath = ".\\Weights\\weights-improvement-{epoch}-singleMidi.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
history = model.fit(network_input, network_output, epochs=10, batch_size=500, callbacks=callbacks_list)
plot_training_curves(history.history);

Epoch 1/10
6453/6453 [==============================] - ETA: 512s - loss: 5.0410 - categorical_crossentropy: 5.0410 - acc: 0.03 - ETA: 465s - loss: 4.9963 - categorical_crossentropy: 4.9963 - acc: 0.03 - ETA: 428s - loss: 4.8538 - categorical_crossentropy: 4.8538 - acc: 0.03 - ETA: 391s - loss: 4.7592 - categorical_crossentropy: 4.7592 - acc: 0.03 - ETA: 346s - loss: 4.6927 - categorical_crossentropy: 4.6927 - acc: 0.03 - ETA: 303s - loss: 4.6513 - categorical_crossentropy: 4.6513 - acc: 0.03 - ETA: 261s - loss: 4.6118 - categorical_crossentropy: 4.6118 - acc: 0.03 - ETA: 217s - loss: 4.5771 - categorical_crossentropy: 4.5771 - acc: 0.03 - ETA: 173s - loss: 4.5608 - categorical_crossentropy: 4.5608 - acc: 0.03 - ETA: 129s - loss: 4.5412 - categorical_crossentropy: 4.5412 - acc: 0.03 - ETA: 84s - loss: 4.5227 - categorical_crossentropy: 4.5227 - acc: 0.0373 - ETA: 40s - loss: 4.5057 - categorical_crossentropy: 4.5057 - acc: 0.036 - 587s - loss: 4.4978 - categorical_crossentropy: 4.4978 

In [ ]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(256))
model.add(Dense(n_vocab,activation='softmax'))
model.load_weights('.\\Weights\\weights-improvement-9-singleMidi.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['categorical_crossentropy','accuracy'])
model.summary()


In [ ]:
# Load the weights to each node

model.evaluate(network_input,network_output,verbose=1)

In [ ]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
pattern=list(pattern)
# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output_500.mid')